In [25]:
import xml.etree.ElementTree as ET
import pandas as pd

In [3]:
graph_xml_path = 'graph-15-08-2.graphml'

tree = ET.parse(graph_xml_path)
root = tree.getroot()

In [5]:
graph_tag = root.findall("{*}graph")

In [7]:
graph_structure = graph_tag[0]

nodes_list = graph_structure.findall("{*}node")

edges_list = graph_structure.findall("{*}edge")

In [9]:
def create_edge_label(polyLineEdge, edge_counter):
    edge_label_attribs = {'alignment':"center", 'configuration':"AutoFlippingLabel", 'distance':"2.0", 'fontFamily':"Dialog", 'fontSize':"12", 
           'fontStyle': "plain", 'hasBackgroundColor':"false", 'hasLineColor':"false", 'height':"20.344114303588867", 'horizontalTextPosition':"center",
           'iconTextGap':"4", 'modelName':"custom", 'preferredPlacement':"anywhere", 'ratio':"0.5", 'textColor':"#000000", 'verticalTextPosition':"bottom", 
           'visible':"true", 'width':"17.72808837890625", 'x':"61.224319634292726", 'xml:space':"preserve", 'y':"-2.9883419125160913"}

    smart_edge_label_model_attribs = {'autoRotationEnabled':"false", 'defaultAngle':"0.0", 'defaultDistance':"10.0"}

    smart_edge_label_model_parameter_attribs = {'angle':"6.283185307179586", 'distance':"12.356235979904756", 'distanceToCenter':"true", 'position':"right", 'ratio':"0.4790840779210871", 'segment':"-1"}

    prefered_placement_descriptor_attribs = {'angle':"0.0", 'angleOffsetOnRightSide':"0", 'angleReference':"absolute", 'angleRotationOnRightSide':"co", 'distance':"-1.0", 'frozen':"true", 'placement':"anywhere", 'side':"anywhere", 'sideReference':"relative_to_edge_flow"}
    
    edge_label_tag = ET.SubElement(polyLineEdge, '{http://www.yworks.com/xml/graphml}EdgeLabel', attrib=edge_label_attribs)

    edge_label_tag.text = str(edge_counter)
    
    label_model = ET.SubElement(edge_label_tag, '{http://www.yworks.com/xml/graphml}LabelModel')
    smart_edge_label_model = ET.SubElement(label_model, '{http://www.yworks.com/xml/graphml}SmartEdgeLabelModel', attrib=smart_edge_label_model_attribs)
    
    model_parameter = ET.SubElement(edge_label_tag, '{http://www.yworks.com/xml/graphml}ModelParameter')
    smart_edge_label_model_parameter = ET.SubElement(model_parameter, '{http://www.yworks.com/xml/graphml}SmartEdgeLabelModelParameter', attribs=smart_edge_label_model_parameter_attribs)
    
    prefered_placement_descriptor = ET.SubElement(edge_label_tag, '{http://www.yworks.com/xml/graphml}PreferedPlacementDescriptor', attribs=prefered_placement_descriptor_attribs)

In [11]:
edge_counter = 1
for edge in edges_list:
    for data in edge:
        for polyLineEdge in data:
            create_edge_label(polyLineEdge, edge_counter)
            edge_counter += 1

In [25]:
output_file_name = graph_xml_path.replace('.graphml', '') + '-with-edge-labels.graphml'

print(output_file_name)

tree.write(output_file_name)

graph-15-08-with-edge-labels.graphml


In [17]:
node_list = []

node_counter = 140
stairs_counter = 1

greyHex = '#808080'
blueHex = '#0000FF'

node_type = {'#808080': 'DecisionPoint', '#FFCC00': 'Build', '#33CCCC': 'Court', '#FF0000':'Restaurant', '#333333': 'Parking', '#0000FF': 'Stairs', '#993366': 'Exit', '#FF00FF': 'Outpatient', '#008000': 'Bench'}

for node in nodes_list:
  for data in node:
    for shapeNode in data:
      node = {}
      isGrey = False
      isBlue = False
      for property in shapeNode:

        property_name = str(property).replace('{http://www.yworks.com/xml/graphml}', '').split()[1].replace("'", '')

        if property_name == 'Fill' and 'color' in property.attrib.keys():
          if property.attrib['color'] == greyHex:
            isGrey = True
          if property.attrib['color'] == blueHex:
            isBlue = True
          node['color'] = property.attrib['color']

        if property_name == 'NodeLabel':
          if isGrey:
            property.text = str(node_counter)
            node_counter += 1
            isGrey = False
          if isBlue:
            property.text = 'E' + str(stairs_counter)
            stairs_counter += 1
            isBlue = False

          node['label'] = property.text

      if 'color' in node.keys():
        node_list.append(node)

for node in node_list:
  node['type'] = node_type[node['color']]

In [19]:
node_list

[{'color': '#FFCC00', 'label': '01', 'type': 'Build'},
 {'color': '#FFCC00', 'label': '02', 'type': 'Build'},
 {'color': '#FFCC00', 'label': '03', 'type': 'Build'},
 {'color': '#FFCC00', 'label': '04', 'type': 'Build'},
 {'color': '#FFCC00', 'label': '05', 'type': 'Build'},
 {'color': '#FFCC00', 'label': '06', 'type': 'Build'},
 {'color': '#FFCC00', 'label': '07', 'type': 'Build'},
 {'color': '#FFCC00', 'label': '10', 'type': 'Build'},
 {'color': '#FFCC00', 'label': '09', 'type': 'Build'},
 {'color': '#FFCC00', 'label': '11', 'type': 'Build'},
 {'color': '#FFCC00', 'label': '12', 'type': 'Build'},
 {'color': '#FFCC00', 'label': '17', 'type': 'Build'},
 {'color': '#FFCC00', 'label': '15', 'type': 'Build'},
 {'color': '#FFCC00', 'label': '16', 'type': 'Build'},
 {'color': '#FFCC00', 'label': '13', 'type': 'Build'},
 {'color': '#FFCC00', 'label': '18', 'type': 'Build'},
 {'color': '#FFCC00', 'label': '19', 'type': 'Build'},
 {'color': '#FFCC00', 'label': '20', 'type': 'Build'},
 {'color':

In [21]:
map_to_df = {'Node': [], 'Type': []}

for node in node_list:
  map_to_df['Node'].append(node['label'])
  map_to_df['Type'].append(node['type'])

In [33]:
df = pd.DataFrame(data = map_to_df)
df = df.sort_values(by=['Type'])
df

,Node,Type
83,Santander,Bench
0,01,Build
52,23,Build
26,48,Build
27,49,Build
...,...,...
202,E23,Stairs
84,E4,Stairs
76,E2,Stairs
210,E25,Stairs


In [37]:
output_file = 'nodes.csv'
df.to_csv(output_file, index=False)

In [89]:
map_edge_to_df = {'Edge': [], 'Distance': []}

for edge in edges_list:
    for data in edge:
        for polyLineEdge in data:
            for property in polyLineEdge:
                tagType = str(property).replace("{http://www.yworks.com/xml/graphml}", '').split(' ')[1].replace("'", '')

                if tagType == 'EdgeLabel':
                    map_edge_to_df['Edge'].append(property.text)
                    map_edge_to_df['Distance'].append(0)

In [91]:
edge_df = pd.DataFrame(data = map_edge_to_df)

In [93]:
output_file = 'edges.csv'
edge_df.to_csv(output_file)